In [0]:
# Importando as bibliotecas necessárias
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Carregar o dataset Iris
iris = load_iris()
X, y = iris.data, iris.target

# Dividir os dados em treino (70%) e teste (30%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Criar o modelo de regressão logística
model = LogisticRegression(max_iter=200)

# Treinar o modelo com os dados de treino
model.fit(X_train, y_train)

# Realizar previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o desempenho com base na acurácia
acc_test = accuracy_score(y_test, y_pred)
print(f"Acurácia no conjunto de teste: {acc_test:.2f}")

# Avaliação adicional com validação cruzada (5 folds)
cv_scores = cross_val_score(model, X, y, cv=5)
print(f"Acurácia média com validação cruzada: {cv_scores.mean():.2f}")


Acurácia no conjunto de teste: 1.00 (100%)
Isso significa que, quando o modelo foi avaliado com dados que ele nunca viu durante o treinamento, todas as amostras foram classificadas corretamente.

Acurácia média com validaação cruzada: 0.97 (97%)
A validação cruzada, ao dividir o dataset em diferentes subconjuntos (folds), fornece uma avaliação mais robusta do desempenho do modelo. Nesse caso, a média de 97% mostra que, em diferentes divisões, o modelo consistentemente teve alta performance.

Dataset Simples:
O dataset Iris é relativamente simples e possui um número reduzido de amostras e características. Isso pode facilitar a obtenção de acurácias elevadas. Em cenários com dados mais complexos, os resultados podem variar e exigir ajustes adicionais no modelo.

Validação Cruzada vs. Conjunto de Teste:

O conjunto de teste pode, em alguns casos, apresentar uma divisão que favorece o desempenho (por sorte, por exemplo), levando a uma acurácia perfeita.
A validação cruzada, por sua vez, avalia o modelo em diferentes divisões do conjunto de dados, oferecendo uma estimativa mais geral do desempenho.
Outras Métricas:
Embora a acurácia seja uma boa métrica para datasets equilibrados, é interessante considerar outras métricas (como precisão, recall e F1-score) em problemas onde as classes podem ser desbalanceadas.

In [0]:
# Importando as bibliotecas necessárias
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Carregar o dataset Iris
iris = load_iris()
X, y = iris.data, iris.target

# Dividir os dados: 70% para treino e 30% para teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Criar um pipeline que inclui escalonamento e o modelo de regressão logística
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(max_iter=200))
])

# --- Etapa 1: Treinamento simples e avaliação com o conjunto de teste ---
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
acc_test = accuracy_score(y_test, y_pred)
print(f"Acurácia no conjunto de teste (modelo base): {acc_test:.2f}")

# --- Etapa 2: Validação cruzada ---
cv_scores = cross_val_score(pipeline, X, y, cv=5)
print(f"Acurácia média com validação cruzada (modelo base): {cv_scores.mean():.2f}")

# --- Etapa 3: Otimização de hiperparâmetros com GridSearchCV ---
# Definir o grid de parâmetros para a regressão logística
param_grid = {
    'model__C': [0.01, 0.1, 1, 10, 100],
    'model__penalty': ['l2'],         # Usando penalização L2 (poderia testar l1 com solver adequado)
    'model__solver': ['lbfgs']         # Solver adequado para a penalização L2
}

# Configurar o GridSearchCV com validação cruzada (5 folds)
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)
print("Melhores parâmetros encontrados:", grid_search.best_params_)

# Avaliar o modelo otimizado no conjunto de teste
y_pred_optimized = grid_search.predict(X_test)
acc_test_optimized = accuracy_score(y_test, y_pred_optimized)
print(f"Acurácia no conjunto de teste (modelo otimizado): {acc_test_optimized:.2f}")

# Opcional: Relatório detalhado e matriz de confusão
print("\nRelatório de classificação (modelo otimizado):")
print(classification_report(y_test, y_pred_optimized))
print("Matriz de confusão (modelo otimizado):")
print(confusion_matrix(y_test, y_pred_optimized))



# Explicação Detalhada

## Treinamento e Avaliação Simples:
- **Pipeline:**  
  Criamos um pipeline que primeiro padroniza as features usando o `StandardScaler` e, em seguida, aplica a regressão logística.
- **Divisão de Dados:**  
  Usamos `train_test_split` para separar os dados em treino e teste.
- **Avaliação:**  
  Após o treinamento, o modelo é avaliado no conjunto de teste e a acurácia é exibida.

## Validação Cruzada:
- Utilizamos `cross_val_score` para realizar a validação cruzada com 5 folds.
- **Folds:**  
  Folds são as subdivisões do conjunto de dados utilizadas na validação cruzada. Por exemplo, com 5 folds, o dataset é dividido em 5 partes iguais e, em cada iteração, 4 partes são usadas para treinamento e 1 para validação. Essa técnica ajuda a garantir que o modelo seja avaliado de forma robusta e consistente em diferentes subconjuntos dos dados.
- Essa abordagem permite avaliar a consistência do modelo em diferentes divisões do dataset, fornecendo uma média da acurácia.

## Otimização de Hiperparâmetros (GridSearchCV):
- **Definição de Parâmetros:**  
  Montamos um grid com diferentes valores para o parâmetro `C`, que controla a regularização do modelo.
- **Busca em Grade:**  
  O `GridSearchCV` testa todas as combinações dos parâmetros definidos usando validação cruzada, identificando a melhor configuração para o modelo.
- **Solver:**  
  O solver é o algoritmo utilizado para otimizar os coeficientes do modelo de regressão logística. No exemplo, usamos o solver `lbfgs`, que é adequado para a penalização L2 e é eficiente para conjuntos de dados pequenos a médios.
- **Avaliação do Modelo Otimizado:**  
  Após encontrar os melhores parâmetros, reavaliamos o modelo no conjunto de teste e geramos:
  - Um **relatório de classificação** que detalha métricas como precisão, recall e F1-score.
  - A **matriz de confusão**, que é uma ferramenta para visualizar o desempenho do modelo. Ela mostra a quantidade de classificações corretas e incorretas para cada classe, onde cada linha representa as instâncias reais e cada coluna representa as previsões do modelo.


# Exemplo de Consumo do Modelo

Após treinar e otimizar o modelo, podemos utilizá-lo para fazer previsões em novos dados. Este exemplo demonstra como consumir o modelo para prever a classe de uma nova amostra.

## Exemplo de Código

```python
import numpy as np

# Exemplo de novo dado com as mesmas features utilizadas no treinamento:
# [comprimento_sépala, largura_sépala, comprimento_pétala, largura_pétala]
novo_exemplo = np.array([[6.0, 3.0, 4.8, 1.8]])

# Realizando a previsão utilizando o modelo otimizado.
# Aqui estamos utilizando o objeto `grid_search` que já contém o pipeline treinado e otimizado.
predicao = grid_search.predict(novo_exemplo)
print("Predição para o novo exemplo:", predicao)


In [0]:
import numpy as np

# Simulação de um novo dado com as mesmas features utilizadas no treinamento:
# [comprimento_sépala, largura_sépala, comprimento_pétala, largura_pétala]
novo_exemplo = np.array([[6.0, 3.0, 4.8, 1.8]])

# Realizando a previsão utilizando o modelo otimizado.
# Neste exemplo, estamos usando o objeto `grid_search` que já contém o pipeline treinado e otimizado.
predicao = grid_search.predict(novo_exemplo)
print("Predição para o novo exemplo:", predicao)


# Possíveis Cenários de Uso

## API REST
- O modelo pode ser integrado a uma API REST que receba dados via requisições HTTP e retorne a previsão, possibilitando a integração com aplicações web ou mobile.

## Processamento Batch
- Em cenários onde é necessário realizar previsões em lote, o modelo pode ser carregado e utilizado para processar grandes volumes de dados periodicamente.

## Monitoramento em Tempo Real
- O modelo pode ser implementado em sistemas que monitoram dados em tempo real, classificando novas amostras à medida que são recebidas.

# Considerações Importantes

## Pré-processamento
- Certifique-se de que os dados de entrada sejam submetidos ao mesmo pré-processamento (como a padronização com `StandardScaler`) que foi utilizado durante o treinamento.

## Persistência do Modelo
- Em ambientes de produção, é recomendado salvar o modelo treinado (por exemplo, com `joblib` ou `pickle`) e carregá-lo conforme necessário, evitando retraining a cada nova previsão.

## Validação
- Teste o modelo com dados reais e monitore sua performance, para garantir que ele continue atendendo aos requisitos do sistema em produção.